In [23]:
import pandas as pd
import numpy as np
import random
import os
import torch
import sys
sys.path.append('../')
from modules import utils, constants

In [24]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [25]:
testing_df = pd.read_csv('../data/18_jan/test_set_basic.csv')
testing_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,0,0,0,1,0,1,0,0,...,1,0,0,1,0,1,0,0,0,1
1,1,0,0,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,1,0,1
2,1,0,1,1,0,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
3,1,1,0,0,0,1,0,0,1,0,...,0,1,1,1,0,0,0,0,0,1
4,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
len(testing_df[testing_df.ana == 0])

3988

In [11]:
testing_df.tail()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
13995,1,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
13996,1,1,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
13997,1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
13998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [12]:
X_test = testing_df.iloc[:, 0:-1]
y_test = testing_df.iloc[:, -1]

X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((14000, 23), (14000,))

In [13]:
dqn_model = utils.load_dqn3('../models/21_jan/dqn_basic_13000000')
test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df = pd.read_csv('../test_dfs/21_jan/dqn_test_df_13000000.csv')
test_df.head()

Using stable baselines 3
Count: 2800
Count: 5600
Count: 8400
Count: 11200
Count: 14000
Testing done.....


,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,score,is_success
0,0.0,17.0,1.0,1.0,1.0,"[cutaneous_lupus, delirium, joint_involvement,...",0.0,14.0,1.0
1,1.0,18.0,1.0,1.0,1.0,"[cutaneous_lupus, delirium, joint_involvement,...",False,17.0,True
2,2.0,19.0,1.0,1.0,1.0,"[cutaneous_lupus, delirium, joint_involvement,...",False,17.0,True
3,3.0,18.0,1.0,1.0,1.0,"[cutaneous_lupus, delirium, joint_involvement,...",False,13.0,True
4,4.0,18.0,1.0,0.0,0.0,"[cutaneous_lupus, delirium, low_c3, joint_invo...",False,2.0,True


In [14]:
test_df.episode_length.min(), test_df.episode_length.max()

(5.0, 24.0)

In [15]:
test_df[test_df.episode_length == test_df.episode_length.min()].iloc[-1]['trajectory']

['cutaneous_lupus', 'delirium', 'joint_involvement', 'low_c3', 'Lupus']

In [16]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

98.12857142857143

In [17]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(18.072928571428573, 0.9625714285714285)

In [18]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(98.12857142857143, 98.12629483856139, 98.12775257258575)

In [19]:
test_df.y_pred.value_counts()

1.0    7241
0.0    6759
Name: y_pred, dtype: int64

In [20]:
acc_bin, prec_bin, recall_bin, f1_bin = utils.test_binary(test_df['y_actual'], test_df['y_pred'])
acc_bin, prec_bin, recall_bin, f1_bin

(98.12857142857143, 98.23228835796161, 98.1509590175245, 98.19160684704582)

In [21]:
test_df[test_df.episode_length==5].iloc[-1]['trajectory']

['cutaneous_lupus', 'delirium', 'joint_involvement', 'low_c3', 'Lupus']

In [22]:
test_df.y_actual.value_counts()

1.0    7247
0.0    6753
Name: y_actual, dtype: int64

#### Saving files

In [ ]:
# test_df.to_csv(f'../test_dfs/21_jan/dqn_test_df_13000000.csv', index=False)
# success_df.to_csv(f'../test_dfs/21_jan/dqn_success_df_13000000.csv', index=False)